In [4]:
!pip install numba==0.48

     |████████████████████████████████| 2.0 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 15.9 MB 6.6 MB/s eta 0:00:01
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.35.0
    Uninstalling llvmlite-0.35.0:
      Successfully uninstalled llvmlite-0.35.0
  Attempting uninstall: numba
    Found existing installation: numba 0.52.0
    Uninstalling numba-0.52.0:
      Successfully uninstalled numba-0.52.0


In [21]:
!pip install regain

In [28]:
%matplotlib inline
import time
import numpy as np

# Import sklearn

In [27]:
from sklearn.covariance import GraphicalLasso
from sklearn.covariance import empirical_covariance

In [30]:
true_cov = np.array([[0.8, 0.0, 0.2, 0.0],
                     [0.0, 0.4, 0.0, 0.0],
                     [0.2, 0.0, 0.3, 0.1],
                     [0.0, 0.0, 0.1, 0.7]])

In [34]:
np.random.seed(243)
X = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=true_cov, size=200)
cov = GraphicalLasso().fit(X)
np.around(cov.covariance_, decimals=3)

array([[ 0.785, -0.039,  0.166,  0.017],
       [-0.039,  0.44 , -0.013,  0.008],
       [ 0.166, -0.013,  0.325,  0.052],
       [ 0.017,  0.008,  0.052,  0.618]])

In [36]:
np.around(cov.location_, decimals=3)

array([ 0.011,  0.044,  0.   , -0.028])

In [38]:
from scipy import linalg
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
import matplotlib.pyplot as plt

# #############################################################################
# Generate the data
n_samples = 60
n_features = 20

prng = np.random.RandomState(1)
prec = make_sparse_spd_matrix(n_features, alpha=.98,
                              smallest_coef=.4,
                              largest_coef=.7,
                              random_state=prng)
cov = linalg.inv(prec)
d = np.sqrt(np.diag(cov))
cov /= d
cov /= d[:, np.newaxis]
prec *= d
prec *= d[:, np.newaxis]
X = prng.multivariate_normal(np.zeros(n_features), cov, size=n_samples)
X -= X.mean(axis=0)
X /= X.std(axis=0)


# Estimate the covariance
emp_cov = np.dot(X.T, X) / n_samples

model = GraphicalLassoCV()
model.fit(X)
cov_ = model.covariance_
prec_ = model.precision_

# lw_cov_, _ = ledoit_wolf(X)
# lw_prec_ = linalg.inv(lw_cov_)

# # #############################################################################
# # Plot the results
# plt.figure(figsize=(10, 6))
# plt.subplots_adjust(left=0.02, right=0.98)

# # plot the covariances
# covs = [('Empirical', emp_cov), ('Ledoit-Wolf', lw_cov_),
#         ('GraphicalLassoCV', cov_), ('True', cov)]
# vmax = cov_.max()
# for i, (name, this_cov) in enumerate(covs):
#     plt.subplot(2, 4, i + 1)
#     plt.imshow(this_cov, interpolation='nearest', vmin=-vmax, vmax=vmax,
#                cmap=plt.cm.RdBu_r)
#     plt.xticks(())
#     plt.yticks(())
#     plt.title('%s covariance' % name)


# # plot the precisions
# precs = [('Empirical', linalg.inv(emp_cov)), ('Ledoit-Wolf', lw_prec_),
#          ('GraphicalLasso', prec_), ('True', prec)]
# vmax = .9 * prec_.max()
# for i, (name, this_prec) in enumerate(precs):
#     ax = plt.subplot(2, 4, i + 5)
#     plt.imshow(np.ma.masked_equal(this_prec, 0),
#                interpolation='nearest', vmin=-vmax, vmax=vmax,
#                cmap=plt.cm.RdBu_r)
#     plt.xticks(())
#     plt.yticks(())
#     plt.title('%s precision' % name)
#     if hasattr(ax, 'set_facecolor'):
#         ax.set_facecolor('.7')
#     else:
#         ax.set_axis_bgcolor('.7')

# # plot the model selection metric
# plt.figure(figsize=(4, 3))
# plt.axes([.2, .15, .75, .7])
# plt.plot(model.cv_results_["alphas"], model.cv_results_["mean_score"], 'o-')
# plt.axvline(model.alpha_, color='.5')
# plt.title('Model selection')
# plt.ylabel('Cross-validation score')
# plt.xlabel('alpha')

# plt.show()

In [40]:
emp_cov.shape

(20, 20)

# Import ADMM SGL

In [15]:
from helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.solver.single_admm_solver import ADMM_SGL

In [44]:
Omega_0 = np.eye(len(emp_cov))
lambda1 = 0.01

In [45]:
sol, info = ADMM_SGL(emp_cov, lambda1, Omega_0 , eps_admm = 1e-5 , verbose = False, latent = False)

ImportError: cannot import name 'cffi_utils' from 'numba.core.typing' (/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/numba/core/typing/__init__.py)

# Import Regain

In [29]:
from regain.covariance import LatentTimeGraphicalLasso

In [ ]:
# p = 100
# N = 1000
# Sigma, Theta = group_power_network(p, K=5, M=2)

# S, samples = sample_covariance_matrix(Sigma, N)
# S = S[0,:,:]
# Theta = Theta[0,:,:]
# lambda1 = 0.01

In [ ]:
singleGL = GraphicalLasso(alpha = lambda1, tol = 1e-6, max_iter = 500, verbose = True)

start = time.time()
model = singleGL.fit(samples[0,:,:].T)
end = time.time(); print(end-start)
 
res_scikit = model.precision_

In [ ]:
Omega_0 = np.eye(p)

